## PropertyHub: 4 Data Engineering

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
import math
import warnings

In [2]:
class DataEngineering():
    def __init__(self):
        self.date = '202306'
        self.room_info_cols = set()
        self.room_amenities_cols = set()
        self.project_details_attributes_cols = set()
        self.facilities_cols = set()
        
        self.nearby_property_type = []
        self.nearby_property_name = []
        self.nearby_property_distance = []
        self.proj_name = []
        self.distance = []
        self.unit = []
        self.error_nearby_prop = 0
        
        self.col_room_info = ['building','facing','no_bedroom','floor','room_size','unit_type']
        self.col_room_amenities = ['air_conditioner','digital_door_lock','furnished','hot_tub','in_room_wifi','kitchen_hood','kitchen_stove','phone','refrigerator','tv','washer','water_heater']
        self.col_facilities = ['bicycle_parking','cctv','convenience_store','fingerprint_access_control','fitness','jacuzzi','keycard_access_control','kids_playground','library','lift','massage_room','park_bbq_areas','parking','restaurant','sauna','security','shuttle_service','steam_room','swimming_pool','wifi']
        self.col_proj_info = ['address','ceiling_height','completed_on','no_building','no_floor','other_name','project_name','total_units','unit_types','utility_fee']
        
        self.check_col_room_info = 1
        self.check_col_room_amenities = 1
        self.check_col_facilities = 1
        self.check_col_proj_info = 1
    
    def import_data(self):
        paths = ['/kaggle/input/ph-3-1-scraping-condo-data',
                '/kaggle/input/ph-3-2-scraping-condo-data',
                '/kaggle/input/ph-3-3-scraping-condo-data',
                '/kaggle/input/ph-3-4-scraping-condo-data',
                '/kaggle/input/ph-3-5-scraping-condo-data',
                '/kaggle/input/ph-3-6-scraping-condo-data']
        file_names = [f"{self.date}_condo_data_1.csv",
                     f"{self.date}_condo_data_2.csv",
                     f"{self.date}_condo_data_3.csv",
                     f"{self.date}_condo_data_4.csv",
                     f"{self.date}_condo_data_5.csv",
                     f"{self.date}_condo_data_6.csv"]
        outputs = [pd.read_csv(f'{paths[i]}/{file_names[i]}') for i in range(len(file_names))]
        
        # join outputs
        output_all = outputs[0]
        for i in range(len(outputs)-1):
            output_all = pd.concat([output_all,outputs[i+1]], axis=0)
        
        # drop empty rows resulting from errors from scraping
        output_all.dropna(subset=['post_name'], axis=0, inplace=True)
        output_all.reset_index(inplace=True, drop=True)
        
        # assign to a class variable
        self.output = output_all
        print('---'*10)
        print('Done importing')
        print(f'Total len: {len(self.output)}')
        print(f"for-rent len: {len(self.output[self.output['rent_flag']==1])}")
        print(f"for-sale len: {len(self.output[self.output['rent_flag']!=1])}")
        print('---'*10)
    
    def find_unique(self):
        # Find unique values to decompose from long format (list form) to wide format
        
        for i in range(len(self.output)):
            self.room_info_cols.update(set(self.output['room_info_header'][i].strip("[]").replace("'","").split(", ")))
            self.room_amenities_cols.update(set(self.output['room_amenities_have'][i].strip("[]").replace("'","").split(", ")))
            self.room_amenities_cols.update(set(self.output['room_amenities_not_have'][i].strip("[]").replace("'","").split(", ")))
            self.project_details_attributes_cols.update(set(self.output['project_details_attributes'][i].strip("[]").replace("'","").split(", ")))
            self.facilities_cols.update(set(self.output['facilities_have'][i].strip("[]").replace("'","").split(", ")))
            self.facilities_cols.update(set(self.output['facilities_not_have'][i].strip("[]").replace("'","").split(", ")))

        # Remove blank cell (if any)
        try:
            self.room_info_cols.remove("")
        except:
            pass

        try:
            self.room_amenities_cols.remove("")
        except:
            pass

        try:
            self.project_details_attributes_cols.remove("")
        except:
            pass

        try:
            self.facilities_cols.remove("")
        except:
            pass
    
    def decompose_info(self):
        ######################################## Decompose room info ########################################
        room_cols_name = sorted(list(self.room_info_cols))
        self.df_room_info = pd.DataFrame({},columns=room_cols_name)
        for i in range(len(self.output)):
            header_list = self.output['room_info_header'][i].strip("[]").replace("'","").split(", ")
            value_list = self.output['room_info_value'][i].strip("[]").replace('"',"'").split("', ")
            temp = np.zeros(len(room_cols_name)).tolist()

            for j, room_col in enumerate(header_list):
                for _, _ in enumerate(room_cols_name):
                    for k in range(len(room_cols_name)):
                        if room_cols_name[k]==room_col:
                            temp[k] = value_list[j].replace("'","")

            for n in range(len(room_cols_name)):
                temp[n] = '' if temp[n]==0 else str(temp[n])

            df_room_temp = pd.DataFrame(temp).T
            df_room_temp.columns = room_cols_name
            self.df_room_info = pd.concat([self.df_room_info,df_room_temp])
        self.df_room_info.reset_index(inplace=True, drop=True)
        self.df_room_info.columns = self.col_room_info
        print('Finish decomposing room info')
        
        ######################################## Decompose room amenities ########################################
        room_amenities_cols_name = sorted(list(self.room_amenities_cols))
        self.df_room_amenities = pd.DataFrame({},columns=room_amenities_cols_name)
        for i in range(len(self.output)):
            haves = self.output['room_amenities_have'][i].strip("[]").replace("'","").split(", ")
            not_haves = self.output['room_amenities_not_have'][i].strip("[]").replace("'","").split(", ")
            temp = np.zeros(len(room_amenities_cols_name)).tolist()

            for j, have in enumerate(haves):
                for _, _ in enumerate(room_amenities_cols_name):
                    for k in range(len(room_amenities_cols_name)):
                        if room_amenities_cols_name[k]==have:
                            temp[k] = 1

            for n in range(len(room_amenities_cols_name)):
                temp[n] = 1 if temp[n]==1 else 0

            df_amen_temp = pd.DataFrame(temp).T
            df_amen_temp.columns = room_amenities_cols_name
            self.df_room_amenities = pd.concat([self.df_room_amenities,df_amen_temp])
        self.df_room_amenities.reset_index(inplace=True, drop=True)
        self.df_room_amenities.columns = self.col_room_amenities
        print('Finish decomposing room amenities')
        
        ######################################## Decompose project facilities ########################################
        facilities_cols_name = sorted(list(self.facilities_cols))
        self.df_facilities = pd.DataFrame({},columns=facilities_cols_name)
        for i in range(len(self.output)):
            haves = self.output['facilities_have'][i].strip("[]").replace("'","").split(", ")
            not_haves = self.output['facilities_not_have'][i].strip("[]").replace("'","").split(", ")
            temp = np.zeros(len(facilities_cols_name)).tolist()

            for j, have in enumerate(haves):
                for _, _ in enumerate(facilities_cols_name):
                    for k in range(len(facilities_cols_name)):
                        if facilities_cols_name[k]==have:
                            temp[k] = 1

            for n in range(len(facilities_cols_name)):
                temp[n] = 1 if temp[n]==1 else 0

            df_fac_temp = pd.DataFrame(temp).T
            df_fac_temp.columns = facilities_cols_name
            self.df_facilities = pd.concat([self.df_facilities,df_fac_temp])
        self.df_facilities.reset_index(inplace=True, drop=True)
        self.df_facilities.columns = self.col_facilities
        print('Finish decomposing project facilities')
        
        ######################################## Decompose project info ########################################
        # Function to decompose project info
        def get_value_list(name,num):
            texts = self.output[name][num].strip("[]").replace('"',"'")
            text_list = []
            index_in = []
            index_out = []
            start_index = 0
            for i,text in enumerate(texts.split(", ")):
                if '[' in text.strip():
                    index_in.append(i)
                if ']' in text.strip():
                    index_out.append(i)

            for i in range(len(index_in)):
                text_list.append(texts[texts.find('[',start_index):texts.find(']',start_index)+1])
                start_index = texts.find(']')+1

            texts_no_brackets = texts
            for i in range(texts.count('[')):
                texts_no_brackets = texts_no_brackets.replace(text_list[i]+',','')

            texts_not_yet_added_list = [t.strip() for t in texts_no_brackets.split(", ")]

            for i in range(len(index_in)):
                if i == 0:
                    texts_not_yet_added_list.insert(index_in[i],text_list[i])
                else:
                    texts_not_yet_added_list.insert(index_in[i]-(index_out[i-1]-index_in[i-1]),text_list[i])

            texts_new = texts_not_yet_added_list
            return texts_new
        
        proj_cols_name = sorted(list(self.project_details_attributes_cols))
        self.df_proj_info = pd.DataFrame({},columns=proj_cols_name)
        for i in range(len(self.output)):
            header_list = self.output['project_details_attributes'][i].strip("[]").replace("'","").split(", ")
            value_list = get_value_list('project_details_values',i)
            temp = np.zeros(len(proj_cols_name)).tolist()

            for j, proj_col in enumerate(header_list):
                for _, _ in enumerate(proj_cols_name):
                    for k in range(len(proj_cols_name)):
                        if proj_cols_name[k]==proj_col:
                            temp[k] = value_list[j].replace("'","")

            for n in range(len(proj_cols_name)):
                temp[n] = '' if temp[n]==0 else str(temp[n])

            df_proj_temp = pd.DataFrame(temp).T
            df_proj_temp.columns = proj_cols_name
            self.df_proj_info = pd.concat([self.df_proj_info,df_proj_temp])
        self.df_proj_info.reset_index(inplace=True, drop=True)
        self.df_proj_info.columns = self.col_proj_info
        print('Finish decomposing project info')
        
        ######################################## Decompose nearby property ########################################
        # Decompose objects into dataframe
        for i in range(len(self.output)):
            for pp_type in self.output['nearby_property_type'][i].strip('[]').split("', "):
                self.nearby_property_type.append(pp_type.replace("'",""))
                self.proj_name.append(self.df_proj_info['project_name'][i])
            for pp_name in self.output['nearby_property_name'][i].strip('[]').replace('"',"'").split("', "):
                self.nearby_property_name.append(pp_name.replace("'",""))
            for pp_dis in self.output['nearby_property_distance'][i].strip('[]').split("', "):
                self.nearby_property_distance.append(pp_dis.replace("'",""))

        for pp_dis in self.nearby_property_distance:
            if pp_dis == '':
                self.distance.append('')
                self.unit.append('km')
            elif pp_dis.find('km') > 0:
                self.distance.append(pp_dis[:pp_dis.find('km')-1])
                self.unit.append('km')
            elif pp_dis.find('m') > 0:
                self.distance.append(pp_dis[:pp_dis.find('m')-1])
                self.unit.append('m')
            else:
                self.error_nearby_prop += 1
        if self.error_nearby_prop != 0:
            print(f'There are {self.error_nearby_prop} errors in decomposing project info !!')
        
        print('Finish decomposing nearby property')
        print('---'*10)
        
        ######################################## Outputing ########################################
        print('Done decomposing data objects into dataframe')
        print(f'{len(self.room_info_cols)} room_info_cols: {sorted(self.room_info_cols)}')
        print(f'{len(self.room_amenities_cols)} room_amenities_cols: {sorted(self.room_amenities_cols)}')
        print(f'{len(self.facilities_cols)} facilities_cols: {sorted(self.facilities_cols)}')
        print(f'{len(self.project_details_attributes_cols)} project_details_attributes_cols: {sorted(self.project_details_attributes_cols)}')
        print('---'*10)
        
    def check_col_info(self): 
        # Check if there is any additional columns
        columns_titles = ['project_name','other_name','address','ceiling_height','completed_on','no_building','no_floor','total_units','unit_types','utility_fee']
        if len(self.room_info_cols) != len(self.col_room_info):
            self.check_col_room_info = 0
            print(f'Room info: expecting {len(self.col_room_info)} cols but getting {len(self.room_info_cols)} cols')
        if len(self.room_amenities_cols) != len(self.col_room_amenities):
            self.check_col_room_amenities = 0
            print(f'Room info: expecting {len(self.col_room_amenities)} cols but getting {len(self.room_amenities_cols)} cols')
        if len(self.facilities_cols) != len(self.col_facilities):
            self.check_col_facilities = 0
            print(f'Room info: expecting {len(self.col_facilities)} cols but getting {len(self.facilities_cols)} cols')
        if len(self.project_details_attributes_cols) != len(self.col_proj_info):
            self.check_col_proj_info = 0
            print(f'Room info: expecting {len(self.col_proj_info)} cols but getting {len(self.project_details_attributes_cols)} cols')
            
        total_check_temp = self.check_col_room_info*self.check_col_room_amenities*self.check_col_facilities*self.check_col_proj_info
        
        if total_check_temp:
            print('There are NO errors in info columns')
        else:
            print('There are errors in info columns !!!')
        print('---'*10)
        return total_check_temp   # 0 indicates error, 1 indicates ok
    
    def create_tables(self):
        # reindex proj_info
        columns_titles_reindex = ['project_name','other_name','address','ceiling_height','completed_on','no_building','no_floor','total_units','unit_types','utility_fee']
        self.df_proj_info = self.df_proj_info.reindex(columns=columns_titles_reindex)
        
        # Create a new date column
        date = []
        for i in range(len(self.output)):
            date.append(datetime.strptime(self.output['last_update_date'][i]+' '+self.output['last_update_time'][i], '%d/%m/%Y %H:%M:%S'))
        self.output.insert(loc=2,column='last_update',value=date)
        
        
        ######################################## Create Room table ########################################
        room_detail = self.output[['rent_flag',
                                    'post_name',
                                    'condo_link',
                                    'last_update',
                                    'poster_name',
                                    'poster_status',
                                    'price',
                                    'price_unit',
                                    'deposit',
                                    'deposit_unit',
                                    'advance_payment',
                                    'advance_payment_unit',
                                    'room_description']]
        self.room = pd.concat([room_detail,self.df_room_info,self.df_room_amenities,self.df_proj_info['project_name']], axis=1)
        print(f"All = {len(self.room)} records")

        # Initial cleaning for duplicates
        # Check if post_name is the same. Then check if price & room size & room type & floor & project_name are the same. If yes, drop (flag 1). If not, remain (flag 0)
        warnings.filterwarnings('ignore')

        check_list = self.room[self.room.duplicated(subset=['post_name'], keep=False)]
        print(f"Same post_name = {len(check_list)} records")
        check_same = []
        already_checked = []
        for i in range(len(check_list)):
            post_name = check_list['post_name'].iloc[i]
            if post_name not in already_checked:
                already_checked.append(post_name)
                sames = check_list[check_list['post_name']==post_name]
                sames.sort_values(by=['price','floor'], inplace=True)
                sames.reset_index(inplace=True, drop=True)
                for j in range(len(sames)-1):
                    if (sames['price'].iloc[j] == sames.drop(range(j+1))['price'].iloc[0]) and (sames['room_size'].iloc[j] == sames.drop(range(j+1))['room_size'].iloc[0]) and (sames['unit_type'].iloc[j] == sames.drop(range(j+1))['unit_type'].iloc[0]) and (sames['floor'].iloc[j] == sames.drop(range(j+1))['floor'].iloc[0]) and (sames['project_name'].iloc[j] == sames.drop(range(j+1))['project_name'].iloc[0]):
                        check_same.append(1);
                    else:
                        check_same.append(0);

        df_check_same = pd.DataFrame(check_same)
        same_list = df_check_same[df_check_same[0]==1].index.values.tolist()
        same_index = []
        for i in range(len(check_list)):
            if i in same_list:
                same_index.append(check_list.iloc[i,:].name)
        print(f"Same post_name,price,room_size,etc = {len(same_index)} records")

        # Drop same post name by index = same_index
        self.room.drop(index=same_index, axis=0, inplace=True)
        self.room.reset_index(inplace=True, drop=True)

        # Insert primary key room_id into Room table
        self.room.reset_index(inplace=True)
        self.room.rename(columns = {'index':'room_id'}, inplace = True)
        self.room['room_id'] += 1
        print(f"Remaining = {len(self.room)} records")
        print('Done creating room table')
        
        
        ######################################## Create Project table ########################################
        df_proj = pd.concat([self.df_proj_info,self.df_facilities],axis=1).drop_duplicates(subset=['project_name'])
        df_proj.sort_values(by='project_name', inplace=True)
        df_proj.reset_index(inplace=True, drop=True)
        df_proj.reset_index(inplace=True)
        df_proj.rename(columns = {'index':'project_id'}, inplace=True)
        df_proj['project_id'] += 1

        # Drop other_name and unit_types columns
        self.project = df_proj.drop(columns=['other_name','unit_types'], axis=1)
        
        # Insert foriegn key project_id in Room table
        proj_id = []
        for name in self.room['project_name']:
            proj_id.append(df_proj[df_proj['project_name']==name]['project_id'].iloc[0])
        self.room.insert(loc=1,column='project_id',value=proj_id)
        print('Done creating project table')
        
        
        ######################################## Create Project_Name table ########################################
        proj_name = []
        proj_other_name = []
        for i,other_name in enumerate(self.df_proj_info['other_name']):
            if other_name.find('[') >= 0:
                for other_name_sub in other_name.strip('[]').split(', '):
                    proj_other_name.append(other_name_sub)
                    proj_name.append(self.df_proj_info['project_name'][i])
            else:
                proj_other_name.append(other_name)
                proj_name.append(self.df_proj_info['project_name'][i])

        self.project_name = pd.DataFrame(proj_name, columns=['project_name'])
        self.project_name['other_name'] = proj_other_name
        self.project_name = self.project_name.drop_duplicates(subset=['project_name','other_name'])
        self.project_name.sort_values(by='project_name', inplace=True)

        # Insert foreign key project_id
        proj_id = []
        for name in self.project_name['project_name']:
            proj_id.append(df_proj[df_proj['project_name']==name]['project_id'].iloc[0])
        self.project_name.insert(loc=0,column='project_id',value=proj_id)

        # Insert primary key project_name_id
        self.project_name.reset_index(inplace=True, drop=True)
        self.project_name.reset_index(inplace=True)
        self.project_name.rename(columns = {'index':'project_name_id'}, inplace = True)
        self.project_name['project_name_id'] += 1
        print('Done creating Project_Name table')
        
        
        ######################################## Create Project_Unit_Type table ########################################
        proj_name_unit = []
        proj_unit_type = []
        for i,unit_type in enumerate(self.df_proj_info['unit_types']):
            if unit_type.find('[') >= 0:
                for unit_type_sub in unit_type.strip('[]').split(', '):
                    proj_unit_type.append(unit_type_sub)
                    proj_name_unit.append(self.df_proj_info['project_name'][i])
            else:
                proj_unit_type.append(unit_type)
                proj_name_unit.append(self.df_proj_info['project_name'][i])

        self.project_unit_type = pd.DataFrame(proj_name_unit, columns=['project_name'])
        self.project_unit_type['unit_type'] = proj_unit_type
        self.project_unit_type = self.project_unit_type.drop_duplicates(subset=['project_name','unit_type'])
        self.project_unit_type.sort_values(by='project_name', inplace=True)

        # Insert foreign key project_id
        proj_id = []
        for name in self.project_unit_type['project_name']:
            proj_id.append(df_proj[df_proj['project_name']==name]['project_id'].iloc[0])
        self.project_unit_type.insert(loc=0,column='project_id',value=proj_id)

        # Insert primary key unit_type_id
        self.project_unit_type.reset_index(inplace=True, drop=True)
        self.project_unit_type.reset_index(inplace=True)
        self.project_unit_type.rename(columns = {'index':'unit_type_id'}, inplace = True)
        self.project_unit_type['unit_type_id'] += 1
        print('Done creating Project_Unit_Type table')
        
        
        ######################################## Create Property table ########################################
        
        self.property_nearby = pd.DataFrame(self.proj_name, columns=['project_name'])
        self.property_nearby['property_type'] = self.nearby_property_type
        self.property_nearby['property_name'] = self.nearby_property_name
        self.property_nearby['property_distance'] = self.distance
        self.property_nearby['property_distance_unit'] = self.unit
        self.property_nearby = self.property_nearby.drop_duplicates(subset=['project_name','property_type','property_name','property_distance','property_distance_unit'])
        self.property_nearby.sort_values(by='project_name', inplace=True)

        # Insert foreign key project_id
        proj_id = []
        for name in self.property_nearby['project_name']:
            proj_id.append(df_proj[df_proj['project_name']==name]['project_id'].iloc[0])
        self.property_nearby.insert(loc=0,column='project_id',value=proj_id)

        # Insert primary key property_id
        self.property_nearby.reset_index(inplace=True, drop=True)
        self.property_nearby.reset_index(inplace=True)
        self.property_nearby.rename(columns = {'index':'property_id'}, inplace = True)
        self.property_nearby['property_id'] += 1
        
        print('Done creating Property table')
        print('---'*10)
        print(f'Room: {len(self.room)} records')
        print(f'Project: {len(self.project)} records')
        print(f'Project Name: {len(self.project_name)} records')
        print(f'Project Unit: {len(self.project_unit_type)} records')
        print(f'Property Nearby: {len(self.property_nearby)} records')
        print('---'*10)
    
    def wrangle_room(self):
        # room_id, project_id, rent_flag, last_update
        self.room = self.room.astype({'room_id': 'int32', 'project_id': 'int32', 'rent_flag': 'float', 'last_update': 'datetime64[ns]'})
        
        # price, price unit
        repl_index = self.room[(self.room['price']=='please') | (self.room['price']=='Please')].index
        self.room.loc[repl_index,'price'] = np.nan
        for i,text in enumerate(self.room['price']):
            try:
                self.room.loc[i,'price'] = text.replace(',','')
            except:
                continue
        self.room = self.room.astype({'price': 'float'})
        
        repl_index = self.room[(self.room['price_unit']=='contact') | (self.room['price_unit']=='Contact')].index
        self.room.loc[repl_index,'price_unit'] = 'please contact'
        
        # deposit, deposit_unit
        repl_index = self.room[(self.room['deposit']=='please') | (self.room['deposit']=='Please')].index
        self.room.loc[repl_index,'deposit'] = np.nan
        self.room['deposit'] = self.room['deposit'].replace('NO_DEPOSIT',0)
        for i,text in enumerate(self.room['deposit']):
            try:
                self.room.loc[i,'deposit'] = text.replace(',','')
            except:
                continue
        self.room = self.room.astype({'deposit': 'float'})
        
        repl_index = self.room[(self.room['deposit_unit']=='contact') | (self.room['deposit_unit']=='Contact')].index
        self.room.loc[repl_index,'deposit_unit'] = 'please contact'
        
        # advance_payment, advance_payment_unit
        repl_index = self.room[(self.room['advance_payment']=='please') | (self.room['advance_payment']=='Please')].index
        self.room.loc[repl_index,'advance_payment'] = np.nan
        self.room['advance_payment'] = self.room['advance_payment'].replace('NO_ADVANCE_PAYMENT',0)
        for i,text in enumerate(self.room['advance_payment']):
            try:
                self.room.loc[i,'advance_payment'] = text.replace(',','')
            except:
                continue
        self.room = self.room.astype({'advance_payment': 'float'})
        
        repl_index = self.room[(self.room['advance_payment_unit']=='contact') | (self.room['advance_payment_unit']=='Contact')].index
        self.room.loc[repl_index,'advance_payment_unit'] = 'please contact'
        
        # building - seems messy and random -> ignore it
        # facing - looks good
        
        # no_bedroom
        for i,text in enumerate(self.room['no_bedroom']):
            try:
                text_truncate = text.replace('Bed','').replace(' room','')[0]
                self.room.loc[i,'no_bedroom'] = text_truncate if text_truncate.isnumeric() else '1'
            except:
                continue
        self.room['no_bedroom'] = self.room['no_bedroom'].replace('',np.nan)
        self.room = self.room.astype({'no_bedroom': 'float'})
        
        # floor
        for i,text in enumerate(self.room['floor']):
            try:
                text_truncate = text.replace('x','0').replace('X','0').strip('flor ')
                if text_truncate[0:2].strip().isnumeric():
                    self.room.loc[i,'floor'] = text_truncate[0:2].strip()
                elif text_truncate[0:1].strip().isnumeric():
                    self.room.loc[i,'floor'] = text_truncate[0:1].strip()
                else:
                    self.room.loc[i,'floor'] = np.nan
            except:
                continue
        self.room['floor'] = self.room['floor'].replace('',np.nan)
        self.room = self.room.astype({'floor': 'float'})
        
        # room_size
        for i,text in enumerate(self.room['room_size']):
            try:
                text_truncate = text.replace(' sq.m.','').strip()
                text_check = text.replace(' sq.m.','').strip(' .')
                if text_check.isnumeric():
                    self.room.loc[i,'room_size'] = text_truncate if float(text_truncate) <= 3000 else np.nan
                else:
                    self.room.loc[i,'room_size'] = np.nan
            except:
                continue
        self.room['room_size'] = self.room['room_size'].replace('',np.nan)
        self.room = self.room.astype({'room_size': 'float'})
        
        # unit_type
        for i,text in enumerate(self.room['unit_type']):
            try:
                if len(text)>20:
                    self.room.loc[i,'unit_type'] = np.nan
            except:
                continue
        
        # Others
        self.room = self.room.astype({'air_conditioner': 'int32',
                                     'digital_door_lock': 'int32',
                                     'furnished': 'int32',
                                     'hot_tub': 'int32',
                                     'in_room_wifi': 'int32',
                                     'kitchen_hood': 'int32',
                                     'kitchen_stove': 'int32',
                                     'phone': 'int32',
                                     'tv': 'int32',
                                     'washer': 'int32',
                                     'water_heater': 'int32'})
        
        print('Finish wrangling room table')
    
    def wrangle_project(self):
        # project_id
        self.project = self.project.astype({'project_id': 'int32'})
        
        # ceiling_height is ok. It has multiple fields
        
        # completed_on
        for i,text in enumerate(self.project['completed_on']):
            try:
                if text.isnumeric():
                    if (float(text) > 1900) & (float(text) < 2500):
                        self.project.loc[i,'completed_on'] = text
                    elif (float(text) >= 2500):
                        self.project.loc[i,'completed_on'] = float(text)-543
                    else:
                        self.project.loc[i,'completed_on'] = np.nan
                else:
                    self.project.loc[i,'completed_on'] = np.nan
            except:
                continue
        self.project = self.project.astype({'completed_on': 'float'})
        
        # no_building
        for i,text in enumerate(self.project['no_building']):
            try:
                if (len(text.strip()) <= 1) & (text.strip()[0:1].isnumeric()):
                    self.project.loc[i,'no_building'] = text.strip()[0:1] if float(text.strip()[0:1])>0 else np.nan
                elif (len(text.strip()) > 1) & (text.strip()[0:2].isnumeric()):
                    self.project.loc[i,'no_building'] = text.strip()[0:2]
                else:
                    self.project.loc[i,'no_building'] = np.nan
            except:
                continue
        self.project = self.project.astype({'no_building': 'float'})
        
        # no_floor
        for i,text in enumerate(self.project['no_floor']):
            try:
                if text.find(',') >= 1:
                    text = text[:text.find(',')]
                if (len(text.strip()) <= 1) & (text.strip()[0:1].isnumeric()):
                    self.project.loc[i,'no_floor'] = text.strip()[0:1] if float(text.strip()[0:1])>0 else np.nan
                elif (len(text.strip()) > 1) & (text.strip()[0:2].isnumeric()):
                    self.project.loc[i,'no_floor'] = text.strip()[0:2]
                else:
                    self.project.loc[i,'no_floor'] = np.nan
            except:
                continue
        self.project = self.project.astype({'no_floor': 'float'})
        
        # total_units
        for i,text in enumerate(self.project['total_units']):
            try:
                text_truncate = text.replace(',','').replace(' rooms','')
                if text_truncate.isnumeric():
                    if float(text_truncate)<=2500:
                        self.project.loc[i,'total_units'] = text_truncate
                    else:
                        self.project.loc[i,'total_units'] = np.nan
                else:
                    self.project.loc[i,'total_units'] = np.nan
            except:
                continue
        self.project = self.project.astype({'total_units': 'float'})
        
        # utility_fee
        for i,text in enumerate(self.project['utility_fee']):
            try:
                if text.find('-') >= 1:
                    text = text[:text.find('-')]
                if text.find(',') >= 1:
                    text = text[:text.find(',')]
                text_truncate = text.replace(' THB/sq.m.','').strip()
                text_check = text.replace(' THB/sq.m.','').strip(' .')
                if text_check.isnumeric():
                    self.project.loc[i,'utility_fee'] = text_truncate if (float(text_truncate) <= 300) & (float(text_truncate) > 0) else np.nan
                else:
                    self.project.loc[i,'utility_fee'] = np.nan
            except:
                continue
        self.project = self.project.astype({'utility_fee': 'float'})
        
        # Others
        self.project = self.project.astype({'bicycle_parking': 'int32',
                                            'cctv': 'int32',
                                            'convenience_store': 'int32',
                                            'fingerprint_access_control': 'int32',
                                            'fitness': 'int32',
                                            'jacuzzi': 'int32',
                                            'keycard_access_control': 'int32',
                                            'kids_playground': 'int32',
                                            'library': 'int32',
                                            'lift': 'int32',
                                            'massage_room': 'int32',
                                            'park_bbq_areas': 'int32',
                                            'parking': 'int32',
                                            'restaurant': 'int32',
                                            'sauna': 'int32',
                                            'security': 'int32',
                                            'shuttle_service': 'int32',
                                            'steam_room': 'int32',
                                            'swimming_pool': 'int32',
                                            'wifi': 'int32'})
        
        print('Finish wrangling project table')
        
    def wrangle_project_name(self):
        print('Finish wrangling project_name table')
    
    def wrangle_project_unit_type(self):
        print('Finish wrangling project_unit_type table')
    
    def wrangle_property_nearby(self):
        pass
        self.property_nearby['property_distance_unit'] = 'km'
        self.property_nearby['property_distance'] = self.property_nearby['property_distance'].replace('',np.nan)
        self.property_nearby = self.property_nearby.astype({'property_distance': 'float'})
        print('Finish wrangling property_nearby table')
        
    def export_tables(self):
        self.room.to_csv(f"{self.date}_room.csv",index=False)
        self.project.to_csv(f"{self.date}_project.csv",index=False)
        self.project_name.to_csv(f"{self.date}_project_name.csv",index=False)
        self.project_unit_type.to_csv(f"{self.date}_project_unit_type.csv",index=False)
        self.property_nearby.to_csv(f"{self.date}_property_nearby.csv",index=False)
        print('---'*10)
        print('Results exported')
        
    def main(self):
        print('Start Data Engineering ...')
        self.import_data()
        self.find_unique()
        self.decompose_info()
        if self.check_col_info():
            self.create_tables()
            print('Start Data Wrangling ...')
            self.wrangle_room()
            self.wrangle_project()
            self.wrangle_project_name()
            self.wrangle_project_unit_type()
            self.wrangle_property_nearby()
            self.export_tables()

In [3]:
DE = DataEngineering()
DE.main()

Start Data Engineering ...
------------------------------
Done importing
Total len: 158429
for-rent len: 105975
for-sale len: 52454
------------------------------
Finish decomposing room info
Finish decomposing room amenities
Finish decomposing project facilities
Finish decomposing project info
Finish decomposing nearby property
------------------------------
Done decomposing data objects into dataframe
6 room_info_cols: ['Building', 'Facing', 'No. of Bed room', 'On Floor', 'Room size', 'Unit type']
12 room_amenities_cols: ['Air Conditioner', 'Digital Door Lock', 'Furnished', 'Hot Tub', 'In-room WIFI', 'Kitchen Hood', 'Kitchen Stove', 'Phone', 'Refrigerator', 'TV', 'Washer', 'Water Heater']
20 facilities_cols: ['Bicycle Parking', 'CCTV', 'Convenience Store', 'Fingerprint Access Control', 'Fitness', 'Jacuzzi', 'Keycard Access Control', 'Kids Playground', 'Library', 'Lift', 'Massage Room', 'Park / BBQ Areas', 'Parking', 'Restaurant', 'Sauna', 'Security', 'Shuttle Service', 'Steam Room', 